<span class="mathmacros" style="display:none;">
$\def\bra#1{\mathinner{\langle{#1}|}}
\def\ket#1{\mathinner{|{#1}\rangle}}
\def\braket#1{\mathinner{\langle{#1}\rangle}}
\newcommand{\mdef}{\overset{\mathrm{def}}{=}}
\newcommand{\bm}{\mathbf}
\newcommand{\inv}[1]{#1^{-1}}   % Inverse Matrix
\newcommand{\invt}[1]{#1^{-T}}  % Inverse Transposed Matrix
\renewcommand{\nl}{\\&\phantom{{}={}}}% Newline In aligned equations
\newcommand{\pfr}[2]{\frac{\pp #1}{\pp #2}}      % Partial derivative
\newcommand{\dfr}[2]{\frac{\dd #1}{\dd #2}}      % Total derivative
\newcommand{\pp}{\partial}
\newcommand{\ee}{\mathrm{e}}
\newcommand{\ii}{\mathrm{i}}
\DeclareMathOperator{\Var}{Var}
\DeclareMathOperator{\det}{det}
\DeclareMathOperator{\tr}{tr}
\DeclareMathOperator{\sgn}{sgn}
\DeclareMathOperator{\adj}{adj}
\DeclareMathOperator{\dd}{d}
\DeclareMathOperator{\rhs}{RHS}
\DeclareMathOperator{\lhs}{LHS}
\newcommand{\nl}{\\&\phantom{={}}}
\DeclareMathOperator{\E}{E}
\DeclareMathOperator{\Cov}{Cov}
\DeclareMathOperator{\Beta}{B}
\DeclareMathOperator{\Bdist}{Beta}$
</span>

### Wigner function
\begin{align}
W(\alpha)&=D(\alpha)PD(-\alpha)\\
D(\alpha)&=\exp[\alpha a^\dagger-\alpha^*a]\\
P&=\exp(\ii\pi N)=(-1)^N\\
\end{align}

It is obvious that $PP^\dagger=P^2=1$. As $[N, a^\dagger]=a^\dagger$, we apply Baker-Hausdroff's formula to get
\begin{align}
Pa^\dagger P&=\exp(\ii \pi N) a^\dagger \exp(-\ii \pi N)=-a^\dagger\\
PaP&=-a\\
PD_zP&=D_{-z}
\end{align}

Finally we have
$$D(\alpha)PD(-\alpha)=PD(-2\alpha)=Pe^{-2|\alpha|^2}e^{-2\alpha a^\dagger}e^{2\alpha^*a},$$
where $e^{-2\alpha a^\dagger}$ is lower triangular and $e^{2\alpha^*a}$ is upper triangular. This expression is  now handy to evaluate.

Coherent states are hyper complete bases:
$$\iint \frac{\mathrm{d} z}{2\pi}\ket{z}\bra{z}$$

$$D(a)\ket{z}=\ket{z+a},\quad P\ket{z}=\ket{-z}$$

$$\exp\left[\ii\oint (ydx-xdy)\right]=\exp[2A\ii]$$

In [5]:
diag([1,2,3], k=-1)

array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0]])

In [8]:
import scipy.linalg as la

In [37]:
%load_ext Cython

In [40]:
?arange

In [61]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def exp_ap_swift(double complex alpha, int n=20):
    cdef np.ndarray[np.double_t, ndim=1] a = np.sqrt(np.arange(1, n, dtype=np.uint))
    cdef np.ndarray[np.complex_t, ndim=2] A = np.eye(n, dtype=np.complex)
    cdef np.ndarray[np.complex_t, ndim=1] invD = alpha/(np.arange(1, n, dtype=np.uint)[::-1])
    cdef int i
    for i in range(1, n):
        A[i, :i] = A[i-1, :i]*(a[i-1]*invD[n-1-i:])
    return A
    

In [39]:
madelung(10)

-1.7475686037718055

In [325]:
from functools import lru_cache

@lru_cache(1)
def FactorialMatrix(n):
    a = np.sqrt(np.arange(1, n))
    A = np.eye(n, dtype='complex')
    invD = 1/(np.arange(1, n)[::-1])
    for i in range(1, n):
        A[i, :i] = A[i-1, :i]*(a[i-1]*invD[n-1-i:])
    return A

In [326]:
def exp_ap_nocache(alpha, n=20):
    '''exponential of creation operator'''
    a = np.sqrt(np.arange(1, n))
    A = np.eye(n, dtype='complex')
    invD = alpha/(np.arange(1, n)[::-1])
    for i in range(1, n):
        A[i, :i] = A[i-1, :i]*(a[i-1]*invD[n-1-i:])
    return A

def exp_ap_cache(alpha, n=20):
    '''exponential of creation operator'''
    Delta = np.clip(arange(n)[:, np.newaxis] - arange(n)[np.newaxis, :], 0, None)
    return alpha**Delta*FactorialMatrix(n)


def exp_ap_expm(alpha, n=20):
    ap = alpha*np.diag(np.sqrt(np.arange(n-1)), k=-1)
    return la.expm(ap)

In [382]:
%time for t in linspace(0, 2*pi, 1001): exp_ap_nocache(exp(1j*t), 100)

CPU times: user 315 ms, sys: 3.38 ms, total: 319 ms
Wall time: 314 ms


In [280]:
B = Delta(200)

In [283]:
%time for t in linspace(0, 2*pi, 1001): 1.1**Delta(200)

CPU times: user 2.07 s, sys: 0 ns, total: 2.07 s
Wall time: 2.07 s


In [74]:
from scipy import special as sp

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [2.        , 1.        , 0.        , 0.        , 0.        ],
       [2.82842712, 2.82842712, 1.        , 0.        , 0.        ],
       [3.26598632, 4.89897949, 3.46410162, 1.        , 0.        ],
       [3.26598632, 6.53197265, 6.92820323, 4.        , 1.        ]])

$$\braket{n|e^{ca^\dagger}|m}=I(n\geq m) c^{n-m}\sqrt{\frac{m!}{n!}}\binom{n}{m}$$

In [73]:
exp_ap_swift(1, 5).real

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [1.        , 1.        , 0.        , 0.        , 0.        ],
       [0.70710678, 1.41421356, 1.        , 0.        , 0.        ],
       [0.40824829, 1.22474487, 1.73205081, 1.        , 0.        ],
       [0.20412415, 0.81649658, 1.73205081, 2.        , 1.        ]])

In [47]:
exp_ap_fast(2, 3)

array([[1.        +0.j, 0.        +0.j, 0.        +0.j],
       [2.        +0.j, 1.        +0.j, 0.        +0.j],
       [2.82842712+0.j, 2.82842712+0.j, 1.        +0.j]])